<a href="https://colab.research.google.com/github/ndtuan10/DeepLearning_CS431.L21.KHCL/blob/main/BaiTap/BaiTap7_Neural_Machine_Translation%2BVie-Eng_dataset/BaiTap7_Neural_Machine_Translation%2BVie-Eng_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BÀI TẬP 7**

**Môn : Các kỹ thuật học sâu và ứng dụng - CS431.L21.KHCL**

In [2]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Neural Machine Translation (NMT) - Translating Vietnamese sentences to English sentences**

## **What are we going to do?**
We will basically create an encoder-decoder LSTM model using [Keras Functional API](https://www.tensorflow.org/alpha/guide/keras/functional) ( with [TensorFlow](https://www.tensorflow.org/) ). We will convert the Vietnamese sentences to English sentences.

Here's an example,

con mèo ngủ giữa những con chó -> the cat sleeps among the dogs  

So, let's get started.



# **I. Preparing the Data**

## **1) Importing the libraries**

We will import TensorFlow and Keras. From Keras, we import various modules which help in building NN layers, preprocess data and construct LSTM models.

In [3]:
%tensorflow_version 2.x

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers , activations , models , preprocessing , utils
import pandas as pd

## **2) Reading the data**


Our dataset which contains more than 7K pairs of Vietnamese-English phrases. This amazing dataset is available at http://www.manythings.org/anki/ and it also other 50+ sets of bilingual sentences. We download the dataset for Vietnamese-English phrases, unzip it and read it using [Pandas](https://pandas.pydata.org/).

In [4]:
'''!wget http://www.manythings.org/anki/vie-eng.zip
!unzip vie-eng.zip'''

'!wget http://www.manythings.org/anki/vie-eng.zip\n!unzip vie-eng.zip'

In [5]:
f = open('/content/drive/MyDrive/Các kỹ thuật học sâu - CS431/Machine Translation/vie-eng/vie.txt','r')
lines = pd.read_table(f , names=[ 'eng' , 'vie' ] )
print(lines.shape)
lines.reset_index( level=0 , inplace=True )
lines.rename( columns={ 'index' : 'eng' , 'eng' : 'vie' , 'vie' : 'c' } , inplace=True )
lines = lines.drop( 'c' , 1 )
lines.head()

(7523, 2)


,eng,vie
0,Run!,Chạy!
1,Help!,Giúp tôi với!
2,Go on.,Tiếp tục đi.
3,Hello!,Chào bạn.
4,Hurry!,Nhanh lên nào!


## **3) Preparing input data for the Encoder ( `encoder_input_data` )**
The Encoder model will be fed input data which are preprocessed Vietnamese sentences. The preprocessing is done as follows :


1.   Tokenizing the Vietnamese sentences from `vie_lines`.
2.   Determining the maximum length of the Vietnamese sentence that's `max_input_length`.
3.   Padding the `tokenized_vie_lines` to the max_input_length.
4.   Determining the vocabulary size ( `num_vie_tokens` ) for Vietnamese words.

In [6]:
vie_lines = list()
for line in lines.vie:
    vie_lines.append( line ) 

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( vie_lines ) 
tokenized_vie_lines = tokenizer.texts_to_sequences( vie_lines ) 

length_list = list()
for token_seq in tokenized_vie_lines:
    length_list.append( len( token_seq ))
max_input_length = np.array( length_list ).max()
print( 'Vietnamese max length is {}'.format( max_input_length ))

padded_vie_lines = preprocessing.sequence.pad_sequences( tokenized_vie_lines , maxlen=max_input_length , padding='post' )
encoder_input_data = np.array( padded_vie_lines )
print( 'Encoder input data shape -> {}'.format( encoder_input_data.shape ))

vie_word_dict = tokenizer.word_index
num_vie_tokens = len( vie_word_dict )+1
print( 'Number of Vietnamese tokens = {}'.format( num_vie_tokens))

Vietnamese max length is 41
Encoder input data shape -> (7523, 41)
Number of Vietnamese tokens = 2359


## **4) Preparing input data for the Decoder ( `decoder_input_data` )**
The Decoder model will be fed the preprocessed English lines. The preprocessing steps are similar to the ones which are above. This one step is carried out before the other steps.


*   Append `<START>` tag at the first position in  each English sentence.
*   Append `<END>` tag at the last position in  each English sentence.

In [7]:
eng_lines = list()
for line in lines.eng:
    eng_lines.append( '<START> ' + line + ' <END>' )  

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( eng_lines ) 
tokenized_eng_lines = tokenizer.texts_to_sequences( eng_lines ) 

length_list = list()
for token_seq in tokenized_eng_lines:
    length_list.append( len( token_seq ))
max_output_length = np.array( length_list ).max()
print( 'English max length is {}'.format( max_output_length ))

padded_eng_lines = preprocessing.sequence.pad_sequences( tokenized_eng_lines , maxlen=max_output_length, padding='post' )
decoder_input_data = np.array( padded_eng_lines )
print( 'Decoder input data shape -> {}'.format( decoder_input_data.shape ))

eng_word_dict = tokenizer.word_index
num_eng_tokens = len( eng_word_dict )+1
print( 'Number of English tokens = {}'.format( num_eng_tokens))


English max length is 34
Decoder input data shape -> (7523, 34)
Number of English tokens = 3710


## **5) Preparing target data for the Decoder ( decoder_target_data )** 

We take a copy of `tokenized_eng_lines` and modify it like this.



1.   We remove the `<start>` tag which we appended earlier. Hence, the word ( which is `<start>` in this case  ) will be removed.
2.   Convert the `padded_eng_lines` ( ones which do not have `<start>` tag ) to one-hot vectors.

For example :

```
 [ '<start>' , 'xin' , 'chào' , '<end>' ]

```

wil become 

```
 [ 'xin' , 'chào' , '<end>' ]

```

In [8]:
decoder_target_data = list()
for token_seq in tokenized_eng_lines:
    decoder_target_data.append( token_seq[ 1 : ] ) 
    
padded_eng_lines = preprocessing.sequence.pad_sequences( decoder_target_data , maxlen=max_output_length, padding='post' )
onehot_eng_lines = utils.to_categorical( padded_eng_lines , num_eng_tokens )
decoder_target_data = np.array( onehot_eng_lines )
print( 'Decoder target data shape -> {}'.format( decoder_target_data.shape ))

Decoder target data shape -> (7523, 34, 3710)


# **II. Defining and Training the models**

## **1) Defining the Encoder-Decoder model**
- The model will have Embedding, LSTM and Dense layers. The basic configuration is as follows.


*   2 Input Layers : One for `encoder_input_data` and another for `decoder_input_data`.
*   Embedding layer : For converting token vectors to fix sized dense vectors. **( Note :  Don't forget the `mask_zero=True` argument here )**
*   LSTM layer : Provide access to Long-Short Term cells.

Working : 

1.   The `encoder_input_data` comes in the Embedding layer (  `encoder_embedding` ). 
2.   The output of the Embedding layer goes to the LSTM cell which produces 2 state vectors ( `h` and `c` which are `encoder_states` )
3.   These states are set in the LSTM cell of the decoder.
4.   The decoder_input_data comes in through the Embedding layer.
5.   The Embeddings goes in LSTM cell ( which had the states ) to produce seqeuences.

In [9]:
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( num_vie_tokens, 256 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 128 , return_state=True  )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( num_eng_tokens, 256 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 128 , return_state=True , return_sequences=True)
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( num_eng_tokens , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 256)    603904      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    949760      input_2[0][0]                    
______________________________________________________________________________________________

## **2) Training the model**
- We train the model for a number of epochs with RMSprop optimizer and categorical crossentropy loss function.

In [10]:
import time
start = time.time()
model.fit([encoder_input_data , decoder_input_data], decoder_target_data, batch_size=250, epochs=100) 
stop = time.time()
time = {stop - start}
print(f"Training time: {time} s") 
model.save( 'model.h5' ) 

Epoch 1/100
31/31 [==============================] - 11s 104ms/step - loss: 1.5353
Epoch 2/100
31/31 [==============================] - 3s 101ms/step - loss: 1.3101
Epoch 3/100
31/31 [==============================] - 3s 100ms/step - loss: 1.2443
Epoch 4/100
31/31 [==============================] - 3s 100ms/step - loss: 1.1909
Epoch 5/100
31/31 [==============================] - 3s 100ms/step - loss: 1.1532
Epoch 6/100
31/31 [==============================] - 3s 99ms/step - loss: 1.1242
Epoch 7/100
31/31 [==============================] - 3s 99ms/step - loss: 1.0941
Epoch 8/100
31/31 [==============================] - 3s 97ms/step - loss: 1.0669
Epoch 9/100
31/31 [==============================] - 3s 99ms/step - loss: 1.0392
Epoch 10/100
31/31 [==============================] - 3s 98ms/step - loss: 1.0114
Epoch 11/100
31/31 [==============================] - 3s 99ms/step - loss: 0.9852
Epoch 12/100
31/31 [==============================] - 3s 101ms/step - loss: 0.9608
Epoch 13/100
31/31

## **3) Inferencing on the models**

In [11]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 128 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 128 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

## **4) Making some translations**


1.   First, we take a Vietnamese sequence and predict the state values using `enc_model`.
2.   We set the state values in the decoder's LSTM.
3.   Then, we generate a sequence which contains the `<start>` element.
4.   We input this sequence in the `dec_model`.
5.   We replace the `<start>` element with the element which was predicted by the `dec_model` and update the state values.
6.   We carry out the above steps iteratively till we hit the `<end>` tag or the maximum sequence length.


In [12]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( vie_word_dict[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=max_input_length , padding='post')

In [ ]:

enc_model , dec_model = make_inference_models()

for epoch in range( encoder_input_data.shape[0] ):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter vie sentence : ' ) ) )
    #states_values = enc_model.predict( encoder_input_data[ epoch ] )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = eng_word_dict['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in eng_word_dict.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > max_output_length:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )


Enter vie sentence : xin chào bạn
 hello may end
Enter vie sentence : bạn có khỏe không
 do you have any choice end
Enter vie sentence : tôi rất thích bạn
 i like oranges very much end
Enter vie sentence : tôi thấy hoa vàng trên cỏ xanh
 i saw it hard end
Enter vie sentence : sự thật luôn chỉ có một
 that's always a bad person end
Enter vie sentence : tiếp tục đi
 keep looking end
Enter vie sentence : chạy
 run end
Enter vie sentence : chạy ngay
 just run end
Enter vie sentence : chạy ngay đi
 go to go home end
Enter vie sentence : nắm lấy tay anh
 it's up to a mistake end


- ***Comment***: We see that the translated words still do not match the content to be translated. The reason is that our dataset is still not enough to be able to learn well.